In [1]:
!nvidia-smi

Tue Jan  9 06:04:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from __future__ import division
from __future__ import print_function

import time

import math
import torch

import numpy as np
import scipy.sparse as sp
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

In [4]:
# layers
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        # This is not a linear layer, but literally has only parameter values.
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters() # weight paremeters initialization

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv) # uniform distribution U(-stdv,stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # matrix multiplication
        support = torch.mm(input, self.weight)
        # matrix multiplication of the sparse matrix adj and the support matrix.
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [5]:
# model

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        # print('X1',x.shape)    # torch.Size([2708, 16])
        # As for the variable called training, whenever the mode is changed by calling the model.train() or model.eval() function, self.training changes to True or False.
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        # print('X2',x.shape)   # torch.Size([2708, 7])

        return F.log_softmax(x, dim=1) # The nll_loss does not have log and softmax applied, hence using log_softmax.

In [6]:
# utils
def encode_onehot(labels):

    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}

    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="/content/drive/MyDrive/GCN_exercise/data/cora/", dataset="cora"):
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    # print('idx_features_labels',idx_features_labels.shape) # (2708, 1435)
    # print('first',idx_features_labels[:,0])                # id
    # print('last',idx_features_labels[:,-1])                # label, e.g. ['Neural_Networks' 'Rule_Learning' 'Reinforcement_Learning' ...'Genetic_Algorithms' 'Case_Based' 'Neural_Networks']

    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])
    # print('features',features.shape)                       # (2708,1433)
    # print('labels',labels.shape)                           # (2708, 7)

    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)

    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)

    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    # |edges| = (edges개수=5429, 2)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32) # |labels| =(2708,8), 2708 = # data

    #  symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    # print('adj',adj.shape) # (2708, 2708)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0])) # Add self loop to adjacency matrix


    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))

    labels = torch.LongTensor(np.where(labels)[1])

    adj = sparse_mx_to_torch_sparse_tensor(adj)


    # train, val, test에 index를 지정하여 semi-supervised(transductive)를 사용하고자 함
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))            # (2708,1)
    r_inv = np.power(rowsum, -1).flatten()  # (2708, )

    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def accuracy(output, labels):

    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [7]:
# Training
import easydict

args = easydict.EasyDict({"no-cuda":False, "fastmode":False, "seed":42, \
                          "epochs":200, "lr":0.01, "weight_decay":5e-4, \
                          "hidden":16, "dropout":0.5, "cuda":True})

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data()
# print('features', features.shape) # torch.Size([2708, 1433])
# print('labels', labels.shape)     # torch.Size([2708])
# print('adj',adj.shape)            # torch.Size([2708, 2708])
# Model and optimizer
model = GCN(nfeat=features.shape[1],          # 1433
            nhid=args.hidden,                 # 16
            nclass=labels.max().item() + 1,   # 7
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)

    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

# Test
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Loading cora dataset...


<ipython-input-6-d28c1bd70d06>:90: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:605.)
  return torch.sparse.FloatTensor(indices, values, shape)


Epoch: 0001 loss_train: 1.9772 acc_train: 0.1429 loss_val: 1.9649 acc_val: 0.1233 time: 1.5560s
Epoch: 0002 loss_train: 1.9572 acc_train: 0.1714 loss_val: 1.9504 acc_val: 0.1533 time: 0.0057s
Epoch: 0003 loss_train: 1.9435 acc_train: 0.1643 loss_val: 1.9357 acc_val: 0.1367 time: 0.0050s
Epoch: 0004 loss_train: 1.9337 acc_train: 0.1929 loss_val: 1.9204 acc_val: 0.1267 time: 0.0050s
Epoch: 0005 loss_train: 1.9164 acc_train: 0.1857 loss_val: 1.9046 acc_val: 0.1267 time: 0.0049s
Epoch: 0006 loss_train: 1.9030 acc_train: 0.2071 loss_val: 1.8888 acc_val: 0.3400 time: 0.0049s
Epoch: 0007 loss_train: 1.8852 acc_train: 0.2071 loss_val: 1.8733 acc_val: 0.3533 time: 0.0050s
Epoch: 0008 loss_train: 1.8691 acc_train: 0.3286 loss_val: 1.8580 acc_val: 0.3500 time: 0.0049s
Epoch: 0009 loss_train: 1.8578 acc_train: 0.3071 loss_val: 1.8428 acc_val: 0.3500 time: 0.0051s
Epoch: 0010 loss_train: 1.8439 acc_train: 0.3071 loss_val: 1.8282 acc_val: 0.3500 time: 0.0051s
Epoch: 0011 loss_train: 1.8287 acc_train